<a href="https://colab.research.google.com/github/whoami-Lory271/thesis-project/blob/Pyra_Encoder/thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbol legend

* B: batch size
* M: number of channel
* P: patch dimension
* N: number of patches
* L: lookback window


# Installations and imports


In [1]:
!pip install pytorch-lightning==2.0.1.post0 --quiet
!pip install einops==0.6.1 --quiet
!pip install ipdb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.1/731.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 874.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import logging
# https://github.com/gotcha/ipdb
import ipdb
import copy
from google.colab import drive
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
# https://theaisummer.com/einsum-attention/
import einops
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
from sklearn.preprocessing import StandardScaler
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import EarlyStopping

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Helper Functions

In [3]:
# setup logger function
def setup_log(self, level):
    log = logging.getLogger(self.__class__.__name__)
    log.setLevel(level)
    return log

In [4]:
# write log on a file
def write_log(a):
    with open("log.txt", 'w') as file:
        for row in a:
            file.write(str(row))
        log.debug("object logged")

In [5]:
def get_patch_num(patch_len, num_var, stride):
    return (max(patch_len, num_var)-patch_len) // stride + 2

In [6]:
# pad array with nan
def pad_nan(arr, left=0, right=0, dim=0):
    # padding the right side
    if left > 0:
        padshape = list(arr.shape)
        padshape[dim] = left
        arr = torch.cat((torch.full(padshape, np.nan).to(arr.device), arr), dim=dim)

    # padding the left side
    if right > 0:
        padshape = list(arr.shape)
        padshape[dim] = right
        arr = torch.cat((arr, torch.full(padshape, np.nan)).to(arr.device), dim=dim)
    return arr

In [7]:
# split with nan
def split_with_nan(x, sections, axis=0):
    assert x.dtype in [np.float16, np.float32, np.float64]
    arrs = np.array_split(x, sections, axis=axis)
    target_length = arrs[0].shape[axis]
    for i in range(len(arrs)):
        arrs[i] = pad_nan_to_target(arrs[i], target_length, axis=axis)
    return arrs

In [8]:
# pad with nan
def pad_nan_to_target(array, target_length, axis=0):
    assert array.dtype in [np.float16, np.float32, np.float64]
    pad_size = target_length - array.shape[axis]
    if pad_size <= 0:
        return array
    npad = [(0, 0)] * array.ndim
    npad[axis] = (0, pad_size)

    return np.pad(array, pad_width=npad, mode='constant', constant_values=np.nan)

In [9]:
def centerize_vary_length_series(x):
    prefix_zeros = np.argmax(~np.isnan(x).all(axis=-1), axis=1)
    suffix_zeros = np.argmax(~np.isnan(x[:, ::-1]).all(axis=-1), axis=1)
    offset = (prefix_zeros + suffix_zeros) // 2 - prefix_zeros
    rows, column_indices = np.ogrid[:x.shape[0], :x.shape[1]]
    offset[offset < 0] += x.shape[1]
    column_indices = column_indices - offset[:, np.newaxis]

    return x[rows, column_indices]

In [10]:
# random seed functions
def seed_everything(seed):
    pl.seed_everything(seed, workers=True)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Constants

In [11]:
# logger
LOG_LEVEL = logging.DEBUG

# datasets
ELECTRICITY = "electricity"

# models
CoPST = "CoPST"

# device

DEVICE = torch.device('cpu')

if torch.cuda.is_available:
    DEVICE = torch.device('cuda')

#paths
ROOT_FOLDER = "/content/drive/MyDrive/Tesi/code"
MODEL_FOLDER = ROOT_FOLDER + "/models"
CHECKPOINT_FOLDER = ROOT_FOLDER + "/checkpoints"
LOGS_FOLDER = ROOT_FOLDER + "/logs"

#hyperparameters

# Train
BATCH_SIZE = 4
STRIDE = 4
PATCH_LEN = 8
L = 32
PATCH_NUM = get_patch_num(PATCH_LEN, L, STRIDE)
M = 370 # TODO: generalize for all datasets
DICT_MOMENTUM_SIZE = 100

# Eval
MAX_TRAIN_LENGTH = 201
PADDING = MAX_TRAIN_LENGTH-1
EVAL_PATCH_NUM = get_patch_num(PATCH_LEN, MAX_TRAIN_LENGTH, STRIDE)
EVAL_BATCH_SIZE = 32


# training
DETERMINISTIC = True
LOAD_MODEL = False

# Logger

In [12]:
# create logger
log = logging.getLogger('APP')
log.setLevel(LOG_LEVEL)
logging.basicConfig(level=LOG_LEVEL)

# # create console handler and set level to debug
# ch = logging.StreamHandler()
# ch.setLevel(logging.INFO)

# # create formatter
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# # add formatter to ch
# ch.setFormatter(formatter)

# # add ch to logger
# logger.addHandler(ch)

# Preprocessing

## Datasets

In [ ]:
datasets_path = {
    ELECTRICITY: ROOT_FOLDER + "/datasets/electricity"
}

datasets_name = {
    ELECTRICITY: "/LD2011_2014.txt"
}
datasets_processed_name = {
    ELECTRICITY: "/electricity.csv" #"/electricity.npy"
}

### Electricity

**Preprocessing**

In [ ]:
# df = pd.read_csv(datasets_path[ELECTRICITY] + datasets_name[ELECTRICITY], sep = ';')
# df.rename(columns={df.columns[0]: 'Date'},inplace=True)
# values = df.values
# values = values[:, 1:].astype(str)
# for i, value in enumerate(values):
#   values[i] = np.char.replace(value, ",", ".")
# values = values.astype(np.float32)
# np.save(datasets_path[ELECTRICITY] + "/electricity", values)

In [ ]:
# # call this function to initialize the csv file
# def electricity_preprocess(path):
#     data_ecl = pd.read_csv(path + '/LD2011_2014.txt', parse_dates=True, sep=';', decimal=',', index_col=0)
#     data_ecl = data_ecl.resample('1h', closed='right').sum()
#     data_ecl = data_ecl.loc[:, data_ecl.cumsum(axis=0).iloc[8920] != 0]  # filter out instances with missing values
#     data_ecl.index = data_ecl.index.rename('date')
#     data_ecl = data_ecl['2012':]
#     data_ecl.to_csv(path + '/electricity.csv')
#     log.info("electriciy.csv created!")

In [ ]:
# electricity_preprocess(datasets_path[ELECTRICITY])

In [ ]:
class ElectricityDatasetENC(Dataset):
    def __init__(self, tensors):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors), "Size mismatch between tensors"
        self.tensors = tensors

    def __getitem__(self, index):
        return tuple(tensor[index] for tensor in self.tensors)

    def __len__(self):
        return self.tensors[0].size(0)

class ElectricityDataset(Dataset):
    def __init__(self, data, eval_mode = False, p = 0.5, multiplier = 10):
        super().__init__()
        self.data = data
        self.eval_mode = eval_mode
        self.p = p
        self.multiplier = multiplier
        self.epsilon = torch.empty(1).normal_(mean = 0, std = 0.5)
        self.N, self.T, self.D = data.shape # num_ts, time, dim

    def __len__(self):
        # return self.data.shape[0] // self.look_window
        return self.data.shape[0] * self.multiplier

    def __getitem__(self, idx):
        """
        out: [B x MAX_TRAIN_LEN x TIME_DIM]
        """
        # # rescale the index
        # start = idx * self.look_window
        # end = start + self.look_window
        # # tale all the channels and skip the date
        # ts = self.data[start:end]
        # # convert in a tensor
        # ts = torch.from_numpy(ts) # [L x M]
        # ts = einops.rearrange(ts, 'l m -> m l')

        # # during the evaluation of the encoder we want the untouched time serie
        # if self.eval_mode:
        #     return ts
        ts = self.data[idx % self.N]

        return self.transform(ts), self.transform(ts)

    def get_len(self):
        return self.__len__()

    # def get_channels(self):
    #     return self.data.iloc[0, 1:].astype(str).str.replace(',', '.').astype('float32').shape[0]

    def transform(self, x):
        return self.jitter(self.shift(self.scale(x)))

    def jitter(self, x):
        if random.random() > self.p:
            return x
        return x + torch.empty(1).normal_(mean = 0, std = 0.5)

    def scale(self, x):
        if random.random() > self.p:
            return x
        return x * self.epsilon

    def shift(self, x):
        if random.random() > self.p:
            return x
        return x + self.epsilon

class ElectricityDataModule(pl.LightningDataModule):
    def __init__(self, path, batch_size, look_window, max_train_length = None, train_size = 0.6, test_size = 0.2):
        super().__init__()
        self.path = path # path to csv file
        self.batch_size = batch_size
        self.max_train_length = max_train_length
        self.train_size = train_size
        self.test_size = test_size
        # self.data = np.load(path)

    # def prepare_data(self):
    #     # download

    def _get_time_features(self,dt):
        return np.stack([
                dt.minute.to_numpy(),
                dt.hour.to_numpy(),
                dt.dayofweek.to_numpy(),
                dt.day.to_numpy(),
                dt.dayofyear.to_numpy(),
                dt.month.to_numpy(),
                dt.weekofyear.to_numpy(),
            ], axis=1).astype(float)

    def _load_forecast_csv(self, path):
        data = pd.read_csv(path, index_col='date', parse_dates=True)
        dt_embed = self._get_time_features(data.index)
        n_covariate_cols = dt_embed.shape[-1]
        data = data.to_numpy()
        # compute slices
        self.train_slice = slice(None, int(self.train_size * len(data)))
        self.valid_slice = slice(int(self.train_size * len(data)), - int(self.test_size * len(data)))
        self.test_slice = slice(- int(self.test_size * len(data)), None)

        return data, dt_embed, n_covariate_cols

    def _scale_and_transform(self, data, dt_embed, n_covariate_cols):
        # scale data
        scaler = StandardScaler()
        scaler.fit(data[self.train_slice])
        data = scaler.transform(data)
        # NUM_ROWS x NUM_FEATURES -> NUM_FEATURES x NUM_ROWS x 1
        data = np.expand_dims(data.T, -1)  # Each variable is an instance rather than a feature
        if n_covariate_cols > 0:
            dt_scaler = StandardScaler().fit(dt_embed[self.train_slice])
            dt_embed = np.expand_dims(dt_scaler.transform(dt_embed), 0)
            data = np.concatenate([np.repeat(dt_embed, data.shape[0], axis=0), data], axis=-1)
        pred_lens = [24, 48, 96, 288, 672]

        return data, scaler, pred_lens, n_covariate_cols

    def _fit_setup(self, train_data):
        if self.max_train_length is not None:
            sections = train_data.shape[1] // self.max_train_length
        if sections >= 2:
            train_data = np.concatenate(split_with_nan(train_data, sections, axis=1), axis=0)

        temporal_missing = np.isnan(train_data).all(axis=-1).any(axis=0)
        if temporal_missing[0] or temporal_missing[-1]:
            train_data = centerize_vary_length_series(train_data)

        train_data = train_data[~np.isnan(train_data).all(axis=2).all(axis=1)]

        multiplier = 1 if train_data.shape[0] >= self.batch_size else math.ceil(self.batch_size / train_data.shape[0])

        return train_data, multiplier


    def setup(self, stage: str):
        # load csv
        data, dt_embed, n_covariate_cols = self._load_forecast_csv(self.path)

        # scale and transform
        data, scaler, pred_lens, n_covariate_cols = self._scale_and_transform(data, dt_embed, n_covariate_cols)

        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            train_data = data[self.train_slice]
            train_data, multiplier = self._fit_setup(train_data)
            # fit setup
            self.train = ElectricityDataset(torch.from_numpy(train_data).to(torch.float), multiplier = multiplier)
            # self.validate = ElectricityDataset(data[self.valid_slice], self.look_window)

        # Assign test dataset for use in dataloader(s)
        # if stage == "test":
        #     self.test = ElectricityDataset(data[self.test_slice], self.look_window)

        # if stage == "predict":

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, drop_last = True)

    # def val_dataloader(self):
    #     return DataLoader(self.validate, batch_size=self.batch_size, drop_last = True)

    # def test_dataloader(self):
    #     return DataLoader(self.test, batch_size=self.batch_size, drop_last = True)

    # def predict_dataloader(self):



In [ ]:
# em = ElectricityDataModule(datasets_path[ELECTRICITY] + datasets_processed_name[ELECTRICITY],
#                       BATCH_SIZE, 10, MAX_TRAIN_LENGTH)
# em.setup("fit")

In [ ]:
# log.debug(em.train.__getitem__(0)[0].shape)

# Models

## Vanilla Transformer Encoder

In [ ]:
# VanillaTransformer encoder

"""
https://arxiv.org/pdf/1706.03762.pdf
"""

class VanillaTransformerEncoder(nn.Module):
    def __init__(self, d_model, n_heads = 16, dropout = 0.2):
        super(VanillaTransformerEncoder, self).__init__()

        self.log = setup_log(self, LOG_LEVEL)

        self.mha = MultiHeadAttention(d_model, n_heads)
        self.norm1 = nn.LayerNorm(d_model) # maybe batch normalization
        self.dropout1 = nn.Dropout(dropout)

        self.pffn = PositionWiseFeedForwardNetwork(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        # new variable because of residual connection
        z = self.mha(x,x,x)
        z = self.dropout1(z)
        z = self.norm1(z + x)

        # set the new value for the residual connection
        x = z
        z = self.pffn(z)
        z = self.dropout2(z)
        return self.norm2(z + x)

"""
ref: https://d2l.ai/chapter_attention-mechanisms-and-transformers/multihead-attention.html
"""
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads = 6):
        super(MultiHeadAttention, self).__init__()

        self.log = setup_log(self, LOG_LEVEL)

        assert d_model % n_heads == 0, "n_heads must be a multiple of d_model"

        self.n_heads = n_heads
        self.d_k = d_model // n_heads

        self.W_q = nn.Linear(d_model, d_model, bias = False)
        self.W_k = nn.Linear(d_model, d_model, bias = False)
        self.W_v = nn.Linear(d_model, d_model, bias = False)
        self.W_o = nn.Linear(d_model, d_model, bias = False)

    # reshape to compute in parallel the several heads
    def reshape_vector(self, x, inverse = False):
        """
        x: [B x N x D] || [B x N x H x DIM]
        output: [B x N x H x DIM] || [B x N x D]
        """
        out = None

        if not inverse:
            out = einops.rearrange(x, 'b n (dim h) -> b h n dim', h=self.n_heads)
        else:
            out = einops.rearrange(x, 'b h n dim -> b n (dim h)')

        return out

    """
    ref: https://machinelearningmastery.com/the-transformer-attention-mechanism/
    """

    def scaled_attention(self, q, k, v, dk):
        """
        q: [B x H x N x DIM], k: [B x H x N x DIM] , v: [B x H x N x DIM]
        output: [B x H x N x DIM]
        """
        sqrt_d_k = math.sqrt(dk)

        # using einsum to perform batch matrix multiplication
        score = einops.einsum(q, k, 'b h n d_k, b h n_1 d_k -> b h n n_1') / sqrt_d_k

        weights = F.softmax(score, dim = -1)

        res = einops.einsum(weights, v, 'b h n n_1, b h n_1 d_k -> b h n d_k')

        return res

    def forward(self, q, k, v, mask = None):
        """
        q, k, v: [B x N x D]
        output: [B x N x D]
        """
        q = self.reshape_vector(self.W_q(q))
        k = self.reshape_vector(self.W_k(k))
        v = self.reshape_vector(self.W_v(v))

        # parallel computation
        out = self.scaled_attention(q, k, v, self.d_k)
        out_concat = self.reshape_vector(out, inverse = True)

        return self.W_o(out_concat)

class PositionWiseFeedForwardNetwork(nn.Module):
    def __init__(self, d_model, d_inner = 512):
        super(PositionWiseFeedForwardNetwork, self).__init__()

        self.log = setup_log(self, LOG_LEVEL)

        self.W_1 = nn.Linear(d_model, d_inner)
        self.act = nn.GELU()
        self.W_2 = nn.Linear(d_inner, d_model)

    def forward(self, x):
        res = x
        x = self.W_1(x)
        x = self.act(x)
        return res + self.W_2(x)

## PyraformerEncoder

# Helper Functions

In [ ]:
def get_mask(input_size, window_size, inner_size, device):
    """Get the attention mask of PAM-Naive"""
    # Get the size of all layers
    all_size = []
    all_size.append(input_size)
    # we split the nodes in according with the number of children
    for i in range(len(window_size)):
        layer_size = math.floor(all_size[i] / window_size[i])
        all_size.append(layer_size)

    # length of the flattened graph
    seq_length = sum(all_size)
    # mask matrix
    mask = torch.zeros(seq_length, seq_length, device=device)

    # get intra-scale mask
    inner_window = inner_size // 2
    for layer_idx in range(len(all_size)):
        start = sum(all_size[:layer_idx])
        for i in range(start, start + all_size[layer_idx]):
            left_side = max(i - inner_window, start)
            right_side = min(i + inner_window + 1, start + all_size[layer_idx])
            mask[i, left_side:right_side] = 1

    # get inter-scale mask
    for layer_idx in range(1, len(all_size)):
        start = sum(all_size[:layer_idx])
        for i in range(start, start + all_size[layer_idx]):
            left_side = (start - all_size[layer_idx - 1]) + (i - start) * window_size[layer_idx - 1]
            if i == ( start + all_size[layer_idx] - 1):
                right_side = start
            else:
                right_side = (start - all_size[layer_idx - 1]) + (i - start + 1) * window_size[layer_idx - 1]
            mask[i, left_side:right_side] = 1
            mask[left_side:right_side, i] = 1

    mask = (1 - mask).bool()

    return mask, all_size

### Model

In [ ]:
class Bottleneck_Construct(nn.Module):
    """Bottleneck convolution CSCM"""
    def __init__(self, d_model, window_size, d_inner):
        super(Bottleneck_Construct, self).__init__()
        if not isinstance(window_size, list):
            self.conv_layers = nn.ModuleList([
                ConvLayer(d_inner, window_size),
                ConvLayer(d_inner, window_size),
                ConvLayer(d_inner, window_size)
                ])
        else:
            self.conv_layers = []
            for i in range(len(window_size)):
                self.conv_layers.append(ConvLayer(d_inner, window_size[i]))
            self.conv_layers = nn.ModuleList(self.conv_layers)
        self.up = Linear(d_inner, d_model)
        self.down = Linear(d_model, d_inner)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, enc_input):

        temp_input = self.down(enc_input).permute(0, 2, 1)
        all_inputs = []
        for i in range(len(self.conv_layers)):
            temp_input = self.conv_layers[i](temp_input)
            all_inputs.append(temp_input)

        all_inputs = torch.cat(all_inputs, dim=2).transpose(1, 2)
        all_inputs = self.up(all_inputs)
        all_inputs = torch.cat([enc_input, all_inputs], dim=1)

        all_inputs = self.norm(all_inputs)

        return all_inputs
""" For Electricity Dataset"""
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
    pe = torch.zeros(max_len, d_model)
    # create a positional array
    position = torch.arange(0, max_len).unsqueeze(1)
    # div term for half of positions
    div_term = torch.pow(10000.0, torch.arange(0, d_model, 2) / d_model)
    # even positions
    pe[:, 0::2] = torch.sin(position * div_term)
    # odd positions
    pe[:, 1::2] = torch.cos(position * div_term)

    pe = pe.unsqueeze(0)
    self.register_buffer('pe', pe)

    # if normalize:
    #     pe = pe - pe.mean()
    #     pe = pe / (pe.std() * 10)

    def forward(self, x):
        """
        output: (1, input_size, d_model)
        """
        return self.pe[:, :x.size(1)]

class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__>='1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                    kernel_size=3, padding=padding, padding_mode='circular')
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight,mode='fan_in',nonlinearity='leaky_relu')

    def forward(self, x):
        """
        x: (batch, input_size, enc_in)
        output: (batch, input_size, d_model)
        """
        x = self.tokenConv(einops.rearrange(x, 'b l e -> b e l')).transpose(1,2)
        return x

class CustomEmbedding(nn.Module):
    def __init__(self, c_in, d_model, temporal_size, seq_num, dropout=0.1):
        super(CustomEmbedding, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = nn.Linear(temporal_size, d_model)
        self.seqid_embedding = nn.Embedding(seq_num, d_model)

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        """
        x: (batch, input_size, enc_in)
        x_mark: (batch, input_size)
        output: (batch, input_size, d_model)
        """
        x = self.value_embedding(x) + self.position_embedding(x) + self.temporal_embedding(x_mark[:, :, :-1])
            + self.seqid_embedding(x_mark[:, :, -1].long())

        return self.dropout(x)

class EncoderLayer(nn.Module):
    """ Compose with two layers """

    def __init__(self, d_model, d_inner, n_head, dropout=0.1, normalize_before=True, q_k_mask=None, k_q_mask=None):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(d_model, n_head)

        self.pos_ffn = PositionWiseFeedForwardNetwork(
            d_model, d_inner)

    def forward(self, enc_input, slf_attn_mask=None):
        enc_output, enc_slf_attn = self.slf_attn(enc_input, enc_input, enc_input, mask=slf_attn_mask)

        enc_output = self.pos_ffn(enc_output)

        return enc_output, enc_slf_attn

class ConvLayer(nn.Module):
    def __init__(self, c_in, window_size):
        super(ConvLayer, self).__init__()
        self.downConv = nn.Conv1d(in_channels=c_in,
                                  out_channels=c_in,
                                  kernel_size=window_size,
                                  stride=window_size)
        self.norm = nn.BatchNorm1d(c_in)
        self.activation = nn.ELU()

    def forward(self, x):
        x = self.downConv(x)
        x = self.norm(x)
        x = self.activation(x)
        return x

class Bottleneck_Construct(nn.Module):
    """Bottleneck convolution CSCM"""
    def __init__(self, d_model, window_size, d_inner):
        super(Bottleneck_Construct, self).__init__()
        if not isinstance(window_size, list):
            self.conv_layers = nn.ModuleList([
                ConvLayer(d_inner, window_size),
                ConvLayer(d_inner, window_size),
                ConvLayer(d_inner, window_size)
                ])
        else:
            self.conv_layers = []
            for i in range(len(window_size)):
                self.conv_layers.append(ConvLayer(d_inner, window_size[i]))
            self.conv_layers = nn.ModuleList(self.conv_layers)
        self.up = Linear(d_inner, d_model)
        self.down = Linear(d_model, d_inner)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, enc_input):
        """
        enc_input: (batch, input_size, d_model)
        output: (batch, graph_size, d_model)
        """

        temp_input = self.down(enc_input).permute(0, 2, 1)
        all_inputs = []
        for i in range(len(self.conv_layers)):
            temp_input = self.conv_layers[i](temp_input)
            all_inputs.append(temp_input)

        all_inputs = torch.cat(all_inputs, dim=2).transpose(1, 2)
        all_inputs = self.up(all_inputs)
        # concat the computed new nodes with the input nodes
        all_inputs = torch.cat([enc_input, all_inputs], dim=1)

        all_inputs = self.norm(all_inputs)

        return all_inputs

class PyraformerEncoder(nn.Module):
    """ A encoder model with self attention mechanism. """

    def __init__(self, d_model = 256, window_size = [4,4,4], inner_size = 3,
                 input_size = 168, d_inner_hid = 512, n_head = 6, n_layer = 4,
                 # Dataloader parameters
                 enc_in = 1, covariate_size = 3, seq_num = 321,
                 CSCM = "Bottleneck_Construct", d_bottleneck = 128,
                 device = 'cpu'):
        super(PyraformerEncoder, self).__init__()
        self.log = setup_log(self, LOG_LEVEL)

        self.d_model = d_model # size of the latent vector
        self.window_size = # The number of children of a parent node
        self.inner_size = inner_size # The number of ajacent nodes
        self.input_size = input_size # length of the sequence
        self.d_inner_hid = d_inner_hid # inner size of the PostitionalFeedForward
        self.n_head = n_head
        self.n_layer = n_layer
        self.enc_in = enc_in
        self.covariate_size = covariate_size # number of temporal covariate
        self.seq_num = seq_num # size of the time series
        self.CSCM = CSCM # called coarser-scale construction module
        self.d_bottleneck = d_bottleneck #
        self.mask, self.all_size = get_mask(self.input_size, self.window_size, self.inner_size, device)
        self.layers = nn.ModuleList([
                EncoderLayer(self.d_model, self.d_inner_hid, self.n_head) for i in range(self. n_layer)
                ])
        self.enc_embedding = CustomEmbedding(self.enc_in, self.d_model, self.covariate_size, self.seq_num)

        self.conv_layers = eval(self.CSCM)(self.d_model, self.window_size, self.d_bottleneck)

    def forward(self, x_enc, x_mark_enc):
        """
        x_enc: (batch, input_size, enc_in) actual time series
        x_mark_enc: (batch, input_size, covariate_size) covariates of the time series
        output:
        """

        seq_enc = self.enc_embedding(x_enc, x_mark_enc)

        # Repeat the mask for all the batch
        mask = self.mask.repeat(len(seq_enc), 1, 1).to(x_enc.device)
        seq_enc = self.conv_layers(seq_enc)

        for i in range(len(self.layers)):
            seq_enc, _ = self.layers[i](seq_enc, mask)

        if self.decoder_type == 'FC':
            indexes = self.indexes.repeat(seq_enc.size(0), 1, 1, seq_enc.size(2)).to(seq_enc.device)
            indexes = indexes.view(seq_enc.size(0), -1, seq_enc.size(2))
            all_enc = torch.gather(seq_enc, 1, indexes)
            seq_enc = all_enc.view(seq_enc.size(0), self.all_size[0], -1)
        elif self.decoder_type == 'attention' and self.truncate:
            seq_enc = seq_enc[:, :self.all_size[0]]

        return seq_enc

## PatchTST

In [ ]:
# Utility functions

def create_patches(xb, patch_len, stride):
    """
    xb -> [B x L x M] // [B x L x M x T]
    output -> [B x N x M x P] // [B x N x M x T x P], N
    """
    _, num_var, _, _ = xb.shape
    # compute number of patches
    patch_num = (max(patch_len, num_var)-patch_len) // stride + 2

    # we repeat the last variable of the sequence to have equal patches
    tail = torch.repeat_interleave(xb[:,-1:,...], stride, dim = 1)
    xb = torch.concatenate((xb, tail), axis = 1)

    # create patches
    xb = xb.unfold(dimension=1, size=patch_len, step=stride)

    assert patch_num == xb.shape[1], f"wrong number of computed patches, expected {patch_num} but computed {xb.shape[1]}"

    return xb, patch_num

"""
ref: https://kazemnejad.com/blog/transformer_architecture_positional_encoding/
"""

def positional_encoding(batch_size, max_len, d_model):
    """
    output
    """
    pe = torch.zeros(batch_size, max_len, d_model)
    # create a positional array
    position = torch.arange(0, max_len).unsqueeze(1)
    # div term for half of positions
    div_term = torch.pow(10000.0, torch.arange(0, d_model, 2) / d_model)
    # even positions
    pe[:, :, 0::2] = torch.sin(position * div_term)
    # odd positions
    pe[:, :, 1::2] = torch.cos(position * div_term)

    # if normalize:
    #     pe = pe - pe.mean()
    #     pe = pe / (pe.std() * 10)

    return nn.parameter.Parameter(pe, requires_grad= False)

In [ ]:
#PatchTST

class PatchTSTEncoder(nn.Module):
    def __init__(self, num_channels, num_var, patch_len, stride, batch_size, time_dimension = 8, d_model = 128, n_layers = 3, n_heads = 16, dropout = 0.2):
        super(PatchTSTEncoder, self).__init__()

        self.log = setup_log(self, LOG_LEVEL)

        self.num_channels = num_channels
        self.patch_num = (max(patch_len, num_var)-patch_len) // stride + 2
        self.patch_len = patch_len
        self.stride = stride
        self.batch_size = batch_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.n_heads = n_heads
        self.dropout = dropout

        # instance normalization
        """
        ref: https://wandb.ai/wandb_fc/Normalization-Series/reports/Instance-Normalization-in-PyTorch-With-Examples---VmlldzoxNDIyNTQx
        """
        self.inst_norm = nn.InstanceNorm2d(num_channels)

        # patch creation
        self.create_patch = create_patches

        # embedding
        self.W_p = nn.Linear(patch_len * time_dimension, d_model, bias = False)

        # positional encoding
        self.W_pos = positional_encoding(batch_size * num_channels, self.patch_num, d_model)

        # dropout
        self.dropout = nn.Dropout(dropout)

        # encoder
        self.encoders = nn.ModuleList([VanillaTransformerEncoder(d_model) for _ in range(n_layers)])

    def forward(self, x):
        """
        x -> [B x L x M] // [(B x M) x MAX_TRAIN_LENGTH x TIME_DIM]
        output -> [(B M) x N x D]
        """
        b_m, _, _ = x
        assert b_m / self.num_channel == self.batch_size, f"invalid fisrt dimension {b_m / self.num_channel} != {self.batch_size}"
        # [(B M) x MAX_TRAIN_LENGTH x TIME_DIM] -> [B x M x MAX_TRAIN_LENGTH x TIME_DIM]
        x = einops.rearrange(x, '(b m) l t -> b m l t', m=self.num_channels)
        # we need to reshape dimensione before apply instance normalization
        x = einops.rearrange(self.inst_norm(x), 'b m l t -> b l m t')

        # create patches
        x, patch_num = self.create_patch(x, self.patch_len, self.stride)

        # x: [B x N x M x T x P]

        assert self.patch_num == patch_num, f"wrong number for patch_num {self.patch_num} != {patch_num}"

        # reshape the tensor from [B x N x M x T x P] -> [(B M) x N x (P T)]
        x = einops.rearrange(x, 'b n m t p -> (b m) n (p t)')
        # now it can be provided to our transformer implementation

        # project into transformer latent space
        x = self.W_p(x) + self.W_pos

        for layer in self.encoders:
            x = layer(x)

        return x



In [ ]:
M = 3
L = 10
T = 8
P = 2
S = 1
B = 4
x = torch.rand((B * M, L, T))
enc = PatchTSTEncoder(M, L, P, S, B)
x = enc(x)
print(x.shape)

torch.Size([12, 10, 128])


In [ ]:
x = torch.rand((2, 3))
print(x)
print(einops.rearrange(x, 'r c -> (r c)'))

tensor([[0.5178, 0.0855, 0.8716],
        [0.1952, 0.4614, 0.8855]])
tensor([0.5178, 0.0855, 0.8716, 0.1952, 0.4614, 0.8855])


## Cost

In [ ]:
#Cost

class Cost(nn.Module):
    def __init__(self, patch_num, batch_size, d_model = 128, d_s = 64, d_t = 64, n_layers = 3, n_heads = 16, dropout = 0.2):
        super(Cost, self).__init__()

        self.log = setup_log(self, LOG_LEVEL)

        # Dropout for seasonal representation output
        self.seasonal_drop = nn.Dropout(0.1)

        # Trend Feature Disentangler
        self.tfd = TrendFeatureDisentangler(d_model, d_t, patch_num)

        # Seasonal Feature Disentangler
        self.sfd = SeasonalFeatureDisentangler(d_model, d_s, patch_num)

    def forward(self, x):
        """
        x: [(B M) x N x D]
        outputs: {[(B M) x N x d_t], [(B M) x N x d_s]}
        """
        out_tfd = self.tfd(x)

        out_svd = self.sfd(x)

        out_svd = self.seasonal_drop(out_svd)

        return out_tfd, out_svd



# Causal Convolution (dilated)

class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1):
        super(CausalConv1d, self).__init__()

        self.log = setup_log(self, LOG_LEVEL)

        self.kernel_size = kernel_size
        pad = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=pad, dilation=dilation)

    def forward(self, x):
        """
        input: [(B M) x N x i_C]
        output: [(B M) x N_out x o_C]
        """
        # we need to reshape before applying the convolution
        x = einops.rearrange(x, 'b n i_c -> b i_c n')
        x = self.conv(x)

        # we need to remove the trailing padding zeros (except for the fist layer) from the values
        if self.kernel_size > 1:
            x = x[...,0:-(self.kernel_size-1)]

        # rearrange to the original shape
        x = einops.rearrange(x, 'b o_c n -> b n o_c')

        return x

# TFD

class TrendFeatureDisentangler(nn.Module):
    def __init__(self, d_model, d_t, patch_num):
        super(TrendFeatureDisentangler, self).__init__()

        self.log = setup_log(self, LOG_LEVEL)
        self.d_model = d_model
        self.d_t = d_t
        self.patch_num = patch_num

        # https://discuss.pytorch.org/t/causal-convolution/3456/3
        # https://arxiv.org/pdf/1609.03499v2.pdf

        # floor(log(N/2)) autoregressive expert
        self.conv_num = math.floor(math.log2(patch_num / 2)) + 1
        self.convolutions = nn.ModuleList([CausalConv1d(d_model, d_t, 2**i) for i in range(self.conv_num)])

    def avg_pooling(self, input):
        """
        input: [LIST x (B M) x N x d_t]
        """
        return einops.reduce(input, 'list b n d_t -> b n d_t', 'mean')

    def forward(self, x):
        """
        x: [(B M) x N x D]
        output: [(B M) x N x d_t]
        """
        batch_size, patch_num, d_model = x.shape

        assert patch_num == self.patch_num and d_model == self.d_model, "wrong input dimensions"

        # create the result tensor
        out = torch.zeros(self.conv_num, batch_size, patch_num, self.d_t, device = x.device)

        for i, conv in enumerate(self.convolutions):
            out[i,...] = conv(x)

        # apply the average pooling operation
        out = self.avg_pooling(out)

        return out

# SVD

class SeasonalFeatureDisentangler(nn.Module):
    def __init__(self, d_model, d_s, patch_num):
        super(SeasonalFeatureDisentangler, self).__init__()

        self.log = setup_log(self, LOG_LEVEL)

        self.patch_num = patch_num

        # number of frequencies after dft
        self.f = patch_num // 2 + 1

        # discrete fast fourier transform, rfft output contains only the positive frequencies below the Nyquist frequency
        self.dft = torch.fft.rfft

        # Learnable Fourier Layer
        self.fl = FourierLayer(self.f, d_model, d_s, patch_num)

        # inverse of discrete fast fourier transform
        self.idft = torch.fft.irfft



    def forward(self, x):
        """
        x: [(B M) x N x D]
        output: [(B M) x N x d_s]
        """
        # we apply dft along the temporal dimension
        x = self.dft(x, dim = 1)

        assert self.f == x.shape[1], "wrong dimension of dft"

        # apply fourier layer
        x = self.fl(x)

        # compute the inverse of dft to come back to time domain
        x = self.idft(x, n = self.patch_num, dim = 1) # pass also the legth in order to avoid odd-length problems

        return x

class FourierLayer(nn.Module):
    def __init__(self, f, d_model, d_s, patch_num):
        super(FourierLayer, self).__init__()

        self.log = setup_log(self, LOG_LEVEL)

        self.f = f
        self.d_model = d_model

        self.A = nn.Parameter(torch.rand((f, d_model, d_s), dtype=torch.cfloat))
        self.B = nn.Parameter(torch.rand((f, d_s), dtype=torch.cfloat))

    def forward(self, x):
        """
        x: [(B M) x F x D]
        out: [(B M) x F x d_s]
        """
        batch_size, f, _ = x.shape

        assert f == self.f, "wrong dimensions of x"

        out = einops.einsum(self.A, x, 'f d d_s, b f d -> b f d_s') + self.B

        return out




## CoPST Encoder

In [ ]:
# %debug
class CoPSTEncoder(nn.Module):
    def __init__(self, patch_num, num_channels, num_var, patch_len, stride,
                 batch_size, d_model = 128, d_s = 64, d_t = 64, n_layers = 3,
                 n_heads = 16, dropout = 0.2):
        super(CoPSTEncoder, self).__init__()
        # self.save_hyperparameters()

        self.batch_size = batch_size
        self.log = setup_log(self, LOG_LEVEL)

        # PatchTST layer (backbone encoder)
        self.ptst = PatchTSTEncoder(num_channels, num_var, patch_len, stride, batch_size)
        assert patch_num == self.ptst.patch_num, f"wrong number of patches ({patch_num} != {self.ptst.patch_num})"
        self.patch_num = self.ptst.patch_num

        # CoST layer (disentangler)
        self.cost = Cost(self.patch_num, batch_size)

    def encode(self, loader, batch_size, ts_l, device, sliding_length=1, padding=200):
        reprs = []

        n_samples, ts_l, _ = data.shape

        org_training = self.training
        self.eval()

        with torch.no_grad():
            output = []
            for batch in tqdm(loader, desc="input encoding"):
                self.log.debug(f"x.shape {x.shape}")
                x = batch[0] # [B x L x M]
                x = x.to(device)
                # self.log.debug(type(x))
                # self.log.debug(f"shape of batch: {x.shape}")

                # if self.batch_size != batch_size:
                #     self.log.debug("different batch size return same batch")
                #     return batch

                for i in range(0, ts_l, sliding_length):
                    l = i - padding # sliding_padding=200
                    r = i + sliding_length
                    # self.log.debug(x.device)
                    x_sliding = pad_nan(
                        x[:, max(l, 0) : min(r, ts_l)],
                        left=-l if l<0 else 0,
                        right=r-ts_l if r>ts_l else 0,
                        dim=1
                    )
                    if x_sliding.shape[1] != padding+1:
                        self.log.debug(f"l: {l}, r: {r}, ts_l: {ts_l}")
                    # self.log.debug(x_sliding.device)
                    # self.log.debug(f"shape of x_sliding: {x_sliding.shape}")
                    # self.log.debug(x_sliding.shape)
                    reprs.append(self.eval_with_pooling(x_sliding))

                out = torch.cat(reprs, dim=1)
                self.log.debug(f"out.shape: {out.shape}")
                output.append(out)

            output = torch.cat(output, dim=0)

        self.train(org_training)
        return output.numpy()

    def eval_with_pooling(self, x):
        out_t, out_s = self(x)
        out = torch.cat([out_t[:, -1], out_s[:, -1]], dim=-1)
        return einops.rearrange(out.cpu(), 'b d -> b () d')


    def forward(self, x):
        """
        x: [B x L x M] // [B x MAX_TRAIN_LENGTH x TIME_DIM]
        outputs: {[(B M) x N x d_t], [(B M) x N x d_s]}
        """
        nan_mask = ~x.isnan().any(axis=-1)
        x[~nan_mask] = 0

        x = self.ptst(x)

        x = self.cost(x)

        return x

In [ ]:
# M = 3
# L = 10
# T = 8
# P = 2
# S = 1
# B = 4
# P_N = get_patch_num(P, L, S)
# x = torch.rand((B * M, L, T))
# enc = CoPSTEncoder(P_N, M, L, P, S, B)
# x = enc.encode()
# print(x[0].shape, x[1].shape)


TypeError: ignored

## CoPST Model

In [ ]:
# Constrastive model similar to MoCo
"""
https://arxiv.org/pdf/1911.05722.pdf
https://github.com/facebookresearch/moco/blob/main/moco/builder.py
"""

class CoPSTModel(pl.LightningModule):
    def __init__(self, patch_num, num_channels, look_window, patch_len, stride, batch_size, max_train_length,
                 comp_dimension = 64, alpha = 5e-4, K = 65536, m = 0.999, T = 0.07,
                 lr = 1e-3, om = 0.9, wd = 1e-4):
        super(CoPSTModel, self).__init__()
        self.save_hyperparameters(ignore=['encoder_q', 'encoder_k'])

        self.max_train_length = max_train_length

        self.K = K
        self.m = m
        self.T = T

        self.alpha = alpha

        self.lr = lr
        self.om = om
        self.wd = wd

        self.encoder_q = CoPSTEncoder(patch_num, num_channels, look_window, patch_len, stride, batch_size)
        self.encoder_k = copy.deepcopy(self.encoder_q)

        self.patch_num = self.encoder_q.patch_num

        # projections head for queries and keyes
        self.head_q = nn.Sequential(
            nn.Linear(comp_dimension, comp_dimension),
            nn.ReLU(),
            nn.Linear(comp_dimension, comp_dimension)
        )
        self.head_k = nn.Sequential(
            nn.Linear(comp_dimension, comp_dimension),
            nn.ReLU(),
            nn.Linear(comp_dimension, comp_dimension)
        )

        # initialize the parameters of the keyes encoder and projection head
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)
            param_k.requires_grad = False # the keyes encoder will be updated by the momentum update

        for param_q, param_k in zip(self.head_q.parameters(), self.head_k.parameters()):
            param_k.data.copy_(param_q.data)
            param_k.requires_grad = False # the head_k will be updated by the momentum update

        # register a dictionary buffer as a queue (decouped from the minibatch size)
        # https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.register_buffer
        self.register_buffer('queue', F.normalize(torch.randn(comp_dimension, K), dim=0))
        self.register_buffer('queue_ptr', torch.zeros(1, dtype=torch.long))

    def configure_optimizers(self):
        # set parameters of SGD
        optimizer = torch.optim.SGD(self.parameters(), lr = self.lr, momentum = self.om, weight_decay = self.wd)
        # cosine annelling is a wrapper for SGD
        cosine_anneling = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer, T_max = 100)
        return optimizer

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Momentum update for key encoder
        """
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1 - self.m)

        for param_q, param_k in zip(self.head_q.parameters(), self.head_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1 - self.m)

    def compute_loss(self, q, k, k_negs):
        # compute logits
        # positive logits: Bx1 (one timestamp as postive)
        l_pos = einops.einsum(q, k, 'b c,b c->b').unsqueeze(-1)
        # negative logits: BxK
        l_neg = einops.einsum(q, k_negs, 'b c,c k->b k')

        # logits: Bx(1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators - first dim of each batch (it will be considered the positive sample)
        # so we can consider this as a classification problem and use the CE
        labels = torch.zeros(logits.shape[0], dtype=torch.long, device = logits.device)
        loss = F.cross_entropy(logits, labels)

        return loss

    def get_polar(self, x):
        mod = x.abs()
        phase = x.angle()

        return (mod, phase)

    def instance_contrastive_loss(self, z1, z2):
        B = z1.shape[0]
        z = torch.cat([z1, z2], dim=0)  # 2B x F x d_s
        z = einops.rearrange(z, 'b f d_s -> f b d_s')  # F x 2B x d_s
        sim = einops.einsum(z, z, 'f b_1 d_s, f b_2 d_s -> f b_1 b_2')  # F x 2B x 2B
        logits = torch.tril(sim, diagonal=-1)[:, :, :-1]  # F x 2B x (2B-1)
        logits += torch.triu(sim, diagonal=1)[:, :, 1:]
        logits = -F.log_softmax(logits, dim=-1)
        # log.debug(logits)

        i = torch.arange(B)
        loss = (logits[:, i, B + i - 1].mean() + logits[:, B + i, i].mean()) / 2
        return loss

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]

        ptr = int(self.queue_ptr)
        assert self.K % batch_size == 0, "K must be a multiple of batch_size"

        # replace keys at ptr (dequeue and enqueue)
        self.queue[:, ptr:ptr + batch_size] = keys.T

        ptr = (ptr + batch_size) % self.K
        self.queue_ptr[0] = ptr

    def training_step(self, batch, batch_idx):
        x_q, x_k = batch

        if self.max_train_length is not None and x_q.size(1) > self.max_train_length:
            window_offset = np.random.randint(x_q.size(1) - self.max_train_length + 1)
            x_q = x_q[:, window_offset : window_offset + self.max_train_length]
            x_k = x_k[:, window_offset : window_offset + self.max_train_length]

        loss = self.forward(x_q, x_k)

        # logs metrics for each training_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    # def validation_step(self, batch, batch_idx):
    #     x_q, x_k = batch
    #     loss = self.forward(x_q, x_k)

    #     # logs metrics for each training_step,
    #     # and the average across the epoch, to the progress bar and logger
    #     self.log("val_loss", loss)
    #     return loss

    # def test_step(self, batch, batch_idx):
    #     x_q, x_k = batch
    #     loss = self.forward(x_q, x_k)

    #     # logs metrics for each training_step,
    #     # and the average across the epoch, to the progress bar and logger
    #     self.log("test_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    #     return loss

    def forward(self, x_q, x_k):
        """
        x_q, x_k: [B x L x M] // [B x MAX_TRAIN_LENGTH x TIME_DIM]
        """
        # select a random timestamp
        rand_idx = np.random.randint(0, self.patch_num)

        # trend and seasonal queries
        q_t, q_s = self.encoder_q(x_q)

        if q_t is not None:
            q_t = F.normalize(self.head_q(q_t[:, rand_idx]), dim=-1)

        # compute key features
        with torch.no_grad():  # no gradient update for keys (momentum update will be used)
            self._momentum_update_key_encoder()  # update key encoder using momentum
            k_t, k_s = self.encoder_k(x_k)
            k_t = F.normalize(self.head_k(k_t[:, rand_idx]), dim=-1)

        loss = 0

        loss += self.compute_loss(q_t, k_t, self.queue.clone().detach())
        self._dequeue_and_enqueue(k_t)

        q_s = F.normalize(q_s, dim=-1)
        _, k_s = self.encoder_q(x_k)
        k_s = F.normalize(k_s, dim=-1)

        # the frequency and phase lost must be computed in the frequency domain
        q_s_freq = torch.fft.rfft(q_s, dim=1)
        k_s_freq = torch.fft.rfft(k_s, dim=1)
        q_s_amp, q_s_phase = self.get_polar(q_s_freq)
        k_s_amp, k_s_phase = self.get_polar(k_s_freq)

        seasonal_loss = self.instance_contrastive_loss(q_s_amp, k_s_amp) + \
                        self.instance_contrastive_loss(q_s_phase,k_s_phase)
        loss += (self.alpha * (seasonal_loss/2))

        return loss

# Forecasting Evaluation

In [ ]:
data = np.load(datasets_path[ELECTRICITY] + datasets_processed_name[ELECTRICITY])
dataset = ElectricityDataset(data, L, True)
loader = DataLoader(dataset, batch_size=EVAL_BATCH_SIZE, drop_last = True)
log.info(f"device: {DEVICE}")
encoder = CoPSTEncoder(EVAL_PATCH_NUM, M, MAX_TRAIN_LENGTH, PATCH_LEN, STRIDE, EVAL_BATCH_SIZE).to(DEVICE)


# callbacks = [TQDMProgressBar(refresh_rate=20)]
# trainer = pl.Trainer(devices=1, accelerator="auto", callbacks=callbacks)

INFO:APP:device: cuda


In [ ]:
encoder.encode(loader, window_len=L, device=DEVICE, padding=PADDING)

input encoding:   7%|▋         | 9/136 [01:30<22:43, 10.73s/it]

In [ ]:
# pred = trainer.predict(encoder, loader)
# if dataset.__len__() % BATCH_SIZE:
#     pred = pred[:-2] # drop last batch
# pred = torch.cat(pred, dim=0)

# Train

## Helper functions

In [ ]:
# train function
def train(batch_size, datamodule, model, model_name, max_epochs = 500, checkpoint_every_n_epochs = 5,
          check_val_every_n_epoch = 5, resume_training = True, load_model = False,
          enable_checkpoint = True, monitor_metric = "val_loss", checkpoint_dir = None, logs_dir = None,
          early_stopping = True, deterministic = False):

    # check monitor metric
    assert monitor_metric in ["train_loss", "vall_loss"], "metric to monitor is invalid"

    # initialize callbacks array
    callbacks = [TQDMProgressBar(refresh_rate=20)]

    # add checkpoints to callbacks
    checkpoint_callback = None
    if enable_checkpoint and checkpoint_dir is not None:
        checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_dir,  monitor = monitor_metric, filename=model_name + '-mnist-{epoch:02d}-{' + monitor_metric + ':.2f}',
                                         save_last =True, every_n_epochs = checkpoint_every_n_epochs, save_on_train_epoch_end = True)
        callbacks.append(checkpoint_callback)

    # add early stopping to the callbacks
    if early_stopping:
        callbacks.append(EarlyStopping(monitor="val_loss", min_delta = 0.1, patience = 3, mode="min", check_on_train_epoch_end = False))

    # define the logger object
    logger = None
    if logs_dir is not None:
        logger = TensorBoardLogger(logs_dir, name=model_name)

    # create the Trainer
    trainer = pl.Trainer(enable_checkpointing=enable_checkpoint, devices=1, accelerator="auto",
                         max_epochs=max_epochs, max_steps = 2, logger=logger, callbacks=callbacks,  ## remove
                         check_val_every_n_epoch = check_val_every_n_epoch, detect_anomaly=True, ## remove
                         deterministic = deterministic)

    ckpt_path = None
    if resume_training:
        ckpt_path = checkpoint_dir + "last"
    trainer.fit(ckpt_path = ckpt_path, model=model, datamodule=datamodule)
    if checkpoint_callback is not None:
        log.info(checkpoint_callback.best_model_path)

## Main

In [ ]:
seed = 1
# initialize dataset
# datamodule = ElectricityDataModule(datasets_path[ELECTRICITY] + datasets_processed_name[ELECTRICITY], BATCH_SIZE, L)
datamodule = ElectricityDataModule(datasets_path[ELECTRICITY] + datasets_processed_name[ELECTRICITY],
                       BATCH_SIZE, MAX_TRAIN_LENGTH)

# set seed if deterministic
if DETERMINISTIC:
    seed_everything(seed)
# initialize model, or load an extisting one
model = CoPSTModel(PATCH_NUM, M, L, PATCH_LEN, STRIDE, BATCH_SIZE, K = BATCH_SIZE * M * DICT_MOMENTUM_SIZE)
if LOAD_MODEL:
    model = CoPSTModel.load_from_checkpoint(CHECKPOINT_FOLDER)

train(BATCH_SIZE, datamodule, model, CoPST, max_epochs = 100, check_val_every_n_epoch = None,
    resume_training = False, monitor_metric = "train_loss", checkpoint_dir = CHECKPOINT_FOLDER,
    logs_dir = LOGS_FOLDER, early_stopping = False, deterministic = DETERMINISTIC)

In [ ]:
# train(BATCH_SIZE, datamodule, model, CoPST, max_epochs = 100, check_val_every_n_epoch = None,
#       resume_training = False, monitor_metric = "train_loss", checkpoint_dir = CHECKPOINT_FOLDER,
#       logs_dir = LOGS_FOLDER, early_stopping = False, deterministic = DETERMINISTIC)